In [168]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import FunctionTransformer

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
ILUMINA_PATH = Path('../data/Ilumina')
ILUMINA_CLEAN_PATH = Path('../data/Ilumina/clean')

NANOPORE_PATH = Path('../data/Nanopore')
NANOPORE_CLEAN_PATH = Path('../data/Nanopore/clean')

In [106]:
columns = [i for i in range(14) if i != 4] #31
SampleInfo = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Sample info + Sperm quality", skipfooter=1, skiprows=1, usecols=columns, keep_default_na=False, 
            na_values=["null", ""])

columns = [i for i in range(31) if i not in range(2,14)]
SpermQuality = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Sample info + Sperm quality", skipfooter=1, skiprows=1, usecols=columns, keep_default_na=False, 
            na_values=["null", ""])

IluminaPylum = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Pylum-level microbiota")
IluminaFamily = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Family-level microbiota")
IluminaGenus = pd.read_excel(ILUMINA_PATH / "Metadata_and_relative_abundance_of_seminal_microbiota_from_idiopathic_infertile_patients_and_donors.xlsx",
            sheet_name="Genus-level microbiota")

NanoporePylum = pd.read_excel(NANOPORE_PATH / "Relative abundance of seminal microbiota from idiopathic infertile patients and donors using MinION sequencing.xlsx",
            sheet_name="Pylum-level microbiota")
NanoporeFamily = pd.read_excel(NANOPORE_PATH / "Relative abundance of seminal microbiota from idiopathic infertile patients and donors using MinION sequencing.xlsx",
            sheet_name="Family-level microbiota")
NanoporeGenus = pd.read_excel(NANOPORE_PATH / "Relative abundance of seminal microbiota from idiopathic infertile patients and donors using MinION sequencing.xlsx",
            sheet_name="Genus-level microbiota")

In [137]:
IluminaPylum["Fertil"] = 1
IluminaPylum.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

IluminaFamily["Fertil"] = 1
IluminaFamily.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

IluminaGenus["Fertil"] = 1
IluminaGenus.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

NanoporePylum["Fertil"] = 1
NanoporePylum.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

NanoporeFamily["Fertil"] = 1
NanoporeFamily.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

NanoporeGenus["Fertil"] = 1
NanoporeGenus.loc[(IluminaPylum.loc[:,"Sample ID"]).str[:3] == "UAB", "Fertil"] = 0

In [141]:

encode =  KBinsDiscretizer(5, encode='ordinal', strategy='uniform', subsample=None, random_state=2707)

transformer = FunctionTransformer(encode.fit_transform)

IluminaPylumCod = IluminaPylum.copy()
IluminaFamilyCod = IluminaFamily.copy()
IluminaGenusCod = IluminaGenus.copy()

IluminaPylumCod.iloc[:,range(1,IluminaPylumCod.shape[1]-1)] = transformer.transform(IluminaPylumCod.iloc[:,range(1,IluminaPylumCod.shape[1]-1)])
IluminaFamilyCod.iloc[:,range(1,IluminaFamilyCod.shape[1]-1)] = transformer.transform(IluminaFamilyCod.iloc[:,range(1,IluminaFamilyCod.shape[1]-1)])
IluminaGenusCod.iloc[:,range(1,IluminaGenusCod.shape[1]-1)] = transformer.transform(IluminaGenusCod.iloc[:,range(1,IluminaGenusCod.shape[1]-1)])

NanoporePylum.iloc[:,range(1,NanoporePylum.shape[1]-1)] = transformer.transform(NanoporePylum.iloc[:,range(1,NanoporePylum.shape[1]-1)])
NanoporeFamily.iloc[:,range(1,NanoporeFamily.shape[1]-1)] = transformer.transform(NanoporeFamily.iloc[:,range(1,NanoporeFamily.shape[1]-1)])
NanoporeGenus.iloc[:,range(1,NanoporeGenus.shape[1]-1)] = transformer.transform(NanoporeGenus.iloc[:,range(1,NanoporeGenus.shape[1]-1)])


In [225]:
encodeOH = KBinsDiscretizer(n_bins=5, encode='onehot-dense', strategy='uniform', subsample=None, random_state=2707)

IluminaPylumCodOH = IluminaPylumCod.copy()
IluminaFamilyCodOH = IluminaFamilyCod.copy()
IluminaGenusCodOH = IluminaGenusCod.copy()

#IluminaPylumCodOH
columns = IluminaPylumCod.iloc[:,range(1,IluminaPylumCod.shape[1]-1)].columns.tolist()
for i in range(len(columns)):
    aux = pd.get_dummies(IluminaPylumCod[columns[i]], prefix=(columns[i] + "_"))
    IluminaPylumCodOH = pd.concat([IluminaPylumCodOH, aux], axis=1)

IluminaPylumCodOH = IluminaPylumCodOH.drop(columns=columns)
fertilCol = IluminaPylumCodOH.pop("Fertil").astype(bool)
IluminaPylumCodOH.insert(IluminaPylumCodOH.shape[1], "Fertil", fertilCol)
IluminaPylumCodOH = IluminaPylumCodOH.set_index('Sample ID')

#IluminaFamilyCodOH
columns = IluminaFamilyCod.iloc[:,range(1,IluminaFamilyCod.shape[1]-1)].columns.tolist()
for i in range(len(columns)):
    aux = pd.get_dummies(IluminaFamilyCod[columns[i]], prefix=(columns[i] + "_"))
    IluminaFamilyCodOH = pd.concat([IluminaFamilyCodOH, aux], axis=1)

IluminaFamilyCodOH = IluminaFamilyCodOH.drop(columns=columns)
fertilCol = IluminaFamilyCodOH.pop("Fertil").astype(bool)
IluminaFamilyCodOH.insert(IluminaFamilyCodOH.shape[1], "Fertil", fertilCol)
IluminaFamilyCodOH = IluminaFamilyCodOH.set_index('Sample ID')

#IluminaGenusCodOH
columns = IluminaGenusCod.iloc[:,range(1,IluminaGenusCod.shape[1]-1)].columns.tolist()
for i in range(len(columns)):
    aux = pd.get_dummies(IluminaGenusCod[columns[i]], prefix=(columns[i] + "_"))
    IluminaGenusCodOH = pd.concat([IluminaGenusCodOH, aux], axis=1)

IluminaGenusCodOH = IluminaGenusCodOH.drop(columns=columns)
fertilCol = IluminaGenusCodOH.pop("Fertil").astype(bool)
IluminaGenusCodOH.insert(IluminaGenusCodOH.shape[1], "Fertil", fertilCol)
IluminaGenusCodOH = IluminaGenusCodOH.set_index('Sample ID')

,Sample ID,Finegoldia__0.0,Finegoldia__1.0,Finegoldia__2.0,Finegoldia__3.0,Finegoldia__4.0,Peptoniphilus__0.0,Peptoniphilus__1.0,Peptoniphilus__2.0,Peptoniphilus__3.0,...,Gordonibacter__0.0,Gordonibacter__3.0,Gordonibacter__4.0,Tardiphaga__0.0,Tardiphaga__1.0,Tardiphaga__4.0,Acidiphilium__0.0,Acidiphilium__1.0,Acidiphilium__4.0,Fertil
0,UAB 17001,False,True,False,False,False,False,False,True,False,...,True,False,False,True,False,False,True,False,False,False
1,UAB 17002,True,False,False,False,False,True,False,False,False,...,True,False,False,True,False,False,True,False,False,False
2,UAB 17003,False,False,True,False,False,False,True,False,False,...,True,False,False,True,False,False,True,False,False,False
3,UAB 17004,True,False,False,False,False,True,False,False,False,...,True,False,False,True,False,False,True,False,False,False
4,UAB 17005,False,False,False,True,False,True,False,False,False,...,True,False,False,True,False,False,True,False,False,False


In [226]:
SampleInfo.to_csv(ILUMINA_CLEAN_PATH / "SampleInfo.csv", index=False, encoding='utf-8')
SpermQuality.to_csv(ILUMINA_CLEAN_PATH / "SpermQuality.csv", index=False, encoding='utf-8')

IluminaPylumCodOH.to_csv(ILUMINA_CLEAN_PATH / "IluminaPylumCodOH.csv", index=False, encoding='utf-8')
IluminaFamilyCodOH.to_csv(ILUMINA_CLEAN_PATH / "IluminaFamilyCodOH.csv", index=False, encoding='utf-8')
IluminaGenusCodOH.to_csv(ILUMINA_CLEAN_PATH / "IluminaGenusCodOH.csv", index=False, encoding='utf-8')
IluminaPylumCod.to_csv(ILUMINA_CLEAN_PATH / "IluminaPylumCod.csv", index=False, encoding='utf-8')
IluminaFamilyCod.to_csv(ILUMINA_CLEAN_PATH / "IluminaFamilyCod.csv", index=False, encoding='utf-8')
IluminaGenusCod.to_csv(ILUMINA_CLEAN_PATH / "IluminaGenusCod.csv", index=False, encoding='utf-8')
IluminaPylum.to_csv(ILUMINA_CLEAN_PATH / "IluminaPylum.csv", index=False, encoding='utf-8')
IluminaFamily.to_csv(ILUMINA_CLEAN_PATH / "IluminaFamily.csv", index=False, encoding='utf-8')
IluminaGenus.to_csv(ILUMINA_CLEAN_PATH / "IluminaGenus.csv", index=False, encoding='utf-8')

NanoporePylum.to_csv(NANOPORE_CLEAN_PATH / "NanoporePylum.csv", index=False, encoding='utf-8')
NanoporeFamily.to_csv(NANOPORE_CLEAN_PATH / "NanoporeFamily.csv", index=False, encoding='utf-8')
NanoporeGenus.to_csv(NANOPORE_CLEAN_PATH / "NanoporeGenus.csv", index=False, encoding='utf-8')

In [224]:
frequent_itemsets = apriori(IluminaPylumCodOH, min_support=0.001, use_colnames=True)
# rules = association_rules(frequent_itemsets, metric="lift")

# rules.head()


d:\Proyectos\BitsxlaMarato2023\.venv\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value UAB 17001